# PiBlocks shows a fun way to compute digits of $\pi$ using collisions

## If you haven't already seen the video...

Start by checking out 3Blue1Brown's video [The Most Unexpected Answer to a Counting Puzzle](https://www.youtube.com/watch?v=HEfHFsfGXjs)

## But wait, there's more!

3Blue1Brown has a few follow-up videos on his channel showing solutions to the problem.

There are many ways to prove that the number of elastic collisions between two masses on a frictionless plane given the ratio of the masses is $\frac{m_2}{m_1}=100^{k-1}, k\in {\mathbb{N}}_1$

## Let's simulate this first (and prove it with math later)

To make this simulation as simple as possible, there is no time or position component needed. **This means we only need to model the collision dynamics! (You'll see it works in the next section).** This is justifiable because the frictionless assumption means that nothing interesting will happen between collisions. Also, even if the blocks are super close to one another, there is still an infinite precision in distance between them and collisions are perfectly elactic (so no matter how close they get, if you zoom in close enough you'll see that even super fast collisions in close proximity have the same properties as slower ones from farther distances).

### Word to the wise - this simulation has exponential time complexity $O(10^k)$ (the number of times the loop runs increases by a factor of $10$ each time you increment $k$ by $1$). Keep $k$ between $1$ and $7$ so as not to inadvertently break anything. I did not add code to protect against this.

**How to use this Notebook:** If you're new to iPython, Jupyter Notebooks, and Binder, a few comments may be helpful here... The file you're looking at was written as a combination of Python, Markdown, LaTeX, and is saved as something called an iPython Notebook (.ipynb). If you're seeing this page, you followed a link I gave you and clicked on the associated .ipynb. The Notebook format is the brainchild of Project Jupyter, and the Binder project lets you view Notebooks interactively using a tiny chunk of free server space you had allocated to you when you opened this file. Binder is what makes this experience interactive (thank you Binder!). As you follow along, you'll see results pre-populated from the last time I ran this before I posted it online. You can run your own experiments by clicking on each line of code (one by one, starting from the top), then clicking the Run button near the top of the page. There are some really nice people out there paying for our ability to use this server space for free, so please be kind and close this page as soon as you're not actively looking at it (this frees up the resources to other Binder users) - you can always come back and run this again! The open source community thanks you =)

***(Step 1)*** First, we'll need a few tools from numpy.

In [1]:
from numpy import array, matmul

**(Step 2)** Next, we want to select a $k$ for this experiment (which is identical to specifying how many digits of $\pi$ we want to calculate)

In [2]:
k = 3

**(Step 3)** Now let's set the masses and velocities $m_1, m_2, v_1, v_2$. $m_1$ is a unit mass, $m_2$ is then $100^{k-1}$ times larger than the unit mass. Mass 1 is the one that lies between Mass 2 and the wall, and its initial velocity is $0$. The wall is the leftmost object (just based on the way I chose to set up this problem), and leftward motion corresponds to negative velocity while rightward motion means positive velocity. Thus, Mass 2 is initialized with unit negative velocity (but you can change this to any negative velocity without impacting the final result (meaning total number of collisions)).

In [3]:
v1, v2, m1, m2 = 0, -1, 1, 100**(k-1)

**(Step 4)** Now we're going to work from the fundamental [physics modeling our elastic collisions](https://en.wikipedia.org/wiki/Elastic_collision#One-dimensional_Newtonian). You'll find these equations on Wikipedia. Let's reformulate these equations using matrices. The A matrix models the collisions, and the B matrix corresponds to Mass 1 bouncing off the wall (since it simply reflects the velocity of Mass 1 from negative to positive).

$A = \begin{bmatrix}\frac{m_1-m_2}{m_1+m_2}& \frac{2 m_2}{m_1+m_2}\\ \frac{2 m_1}{m_1+m_2}&\frac{m_2-m_1}{m_1+m_2}\end{bmatrix},  B = \begin{bmatrix}-1&0\\ 0&1\end{bmatrix}, v = \begin{bmatrix}v_1\\v_2\end{bmatrix}, u = \begin{bmatrix}u_1\\u_2\end{bmatrix}$

In [4]:
A = array([[(m1-m2),(2*m2)],[(2*m1),(m2-m1)]])/(m1+m2)
B = array([[-1,0],[0,1]])

**(Step 5)** There are no collisions at the outset of this experiment, so let's initialize the collisions counter to $0$.

In [5]:
collisions = 0

**(Step 6)** Here is the meat of the simulation. What you'll see is a while loop, and the loop will keep running over and over again (incrementing the collisions counter by $+1$ each time) until the "exit conditions" are reached. The exit conditions simply state that Mass 1 and Mass 2 are both moving away from the wall, and Mass 2 must be moving at least as fast as Mass 1 so the masses themselves will never collide with each other or with the wall again. Additionally, because Mass 1 has an initial velocity of 0, and Mass 2 has some initial negative velocity, the first collision will always happen between Mass 2 and Mass 1 (so the $A$ matrix models the result of this collision as $v = Au$). You'll notice from the problem statement that Mass 1 and Mass 2 will never collide twice back-to-back. Because of this, after a Mass 1 Mass 2 collision, there will either be a collision with the wall or no collisions will ever happen again. Similarly, when Mass 1 will never collide with the wall twice in a row. After Mass 1 hits the wall (modeled as $v = Bu$), it will either collide with Mass 2 or no collisions will happen ever again. So we get this toggling action between the $A$ matrix and $B$ matrix. Very conveniently, the $A$ matrix models all odd numbered collisions, and the $B$ matrix models all even numbered collisions. The code selects this by using the modulo operation (%) which gives the remainder of an integer division (in our case, we mod by $2$, giving either $0$ or $1$ as the result which tells us whether we should model the next collision using the $A$ or $B$ matrix).

A few notes on the implementation: in Python, the first element of an array has index $0$. We originally implicitly defined $v_1, v_2$ as "number" types, which are scalars. In order to use our $A$ and $B$ matrices, we use the $array()$ function so that we can leverage matrix multiplication functionality with $matmul()$. After this, the results $v[0]$ and $v[1]$ have "array" types, but we want them to be scalars (so as not to break the code - the while loop expects scalar "number" types when evaluating the exit conditions). The $float()$ operator does something called "casting" where the array type is changed to a scalar "float" type. If you're unfamiliar with casting, the intuition behind it is fairly straightforward. A quick google search should clear this up if you're not comfortable with what's going on in the code below.

In [6]:
while not((v1>=0)&(v2>=0)&(v2>=v1)):
    v = array([[v1],[v2]])
    if (collisions%2)<1:
        v = matmul(A,v)
    else:
        v = matmul(B,v)
    v1, v2 = float(v[0]), float(v[1])
    collisions += 1

**(Step 7)** Now that the simulation is done, we just need to print the result. As expected, we get $k$ digits of pi. More specifically, $\pi \approx \frac{collisions}{10^{(k-1)}}$

In [7]:
print(collisions)

314


## There you have it! This Notebook was brought to you by Alex Augenstein and Rich Sofranko. Rich is finishing the write-up on our mathematical proof. I'll either re-create the results below or link to it in a public DropBox (or similar). Thanks for reading!